# Import statements

In [1]:
# Import Dependencies
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
from sklearn.metrics import roc_auc_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA 
from sklearn.svm import LinearSVC
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import mutual_info_classif as MIC
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from sklearn.feature_selection import SelectKBest, SelectPercentile
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import f1_score
from sklearn.model_selection import cross_validate
from sklearn.tree import DecisionTreeClassifier 
from sklearn.linear_model import Perceptron
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
import skopt
from hyperopt import hp

import warnings
warnings.filterwarnings("ignore")

# Loading the Dataset

In [2]:
# Load Dataset
df = pd.read_csv('final_dataset.csv')
df.shape

(5856, 95)

# Train Test Split

In [3]:
X_train, X_test, y_train, y_test = train_test_split(
    df.drop(labels=['Label','Image_ID'], axis=1),
    df['Label'],
    test_size=0.2,
    random_state=23)

X_train.shape, X_test.shape

((4684, 93), (1172, 93))

In [4]:
from sklearn.preprocessing import StandardScaler
def scale_data(dataset):
    scaler = StandardScaler()
    scaled_data = scaler.fit_transform(dataset)
    return scaled_data

In [5]:
scaled_train_set = scale_data(X_train)

scaled_test_set = scale_data(X_test)

In [6]:
def cross_validation(model, _X, _y, _cv=5):
    _scoring = ['f1']
    results = cross_validate(estimator=model,
                             X=_X,
                             y=_y,
                             cv=_cv,
                             scoring=_scoring,
                             return_train_score=True)
      
    return {
              "Mean Validation F1 Score": results['test_f1'].mean()
              }

# Classifier Models

# Logistic Regression

In [7]:
LogisticReg = LogisticRegression(max_iter = 3000000)
LogisticReg.fit(scaled_train_set,y_train)

LogisticRegression(max_iter=3000000)

### Base Model Score

In [8]:
LogisticReg.score(scaled_test_set,y_test)

0.8907849829351536

### Parameters for Logistic Regression Model

In [9]:
LogisticReg.get_params()

{'C': 1.0,
 'class_weight': None,
 'dual': False,
 'fit_intercept': True,
 'intercept_scaling': 1,
 'l1_ratio': None,
 'max_iter': 3000000,
 'multi_class': 'auto',
 'n_jobs': None,
 'penalty': 'l2',
 'random_state': None,
 'solver': 'lbfgs',
 'tol': 0.0001,
 'verbose': 0,
 'warm_start': False}

### HyperParameter Tuning for Logistic Regression Model
* RandomSearch CV method
* GridSearch CV method

In [11]:
LRgrid = {"C":[100, 10, 1.0, 0.1, 0.01],
          "class_weight":[None,'balanced'],
          "penalty":['l2','l1',None,'elasticnet'],
          "solver":['newton-cg', 'newton-cholesky','lbfgs','liblinear','sag','saga']
          }
RCV_LRReg = RandomizedSearchCV(estimator = LogisticReg,
                              param_distributions = LRgrid,
                              n_iter = 20,
                              scoring = 'f1',
                              n_jobs = -1,
                              cv = 5)
RCV_LRReg.fit(scaled_train_set,y_train)

KeyboardInterrupt: 

In [82]:
RCV_LRReg.best_params_

{'solver': 'liblinear', 'penalty': 'l1', 'class_weight': 'balanced', 'C': 100}

In [85]:
RCV_LRReg.score(scaled_test_set,y_test)

0.8859813084112149

In [41]:
LRgrid_ = {'C':[1.0,10.0],
           'penalty':['l1','l2','elasticnet',None],
           'class_weight':[None,'balanced'],
           'solver':['newton-cg','lbfgs','liblinear']
           }
GCV_LRReg = GridSearchCV(estimator = LogisticReg,
                         param_grid = LRgrid_,
                         scoring = 'f1',
                         n_jobs = -1,
                         cv = 5)
GCV_LRReg.fit(scaled_train_set,y_train)

GridSearchCV(cv=5, estimator=LogisticRegression(max_iter=3000000), n_jobs=-1,
             param_grid={'C': [1.0, 10.0], 'class_weight': [None, 'balanced'],
                         'penalty': ['l1', 'l2', 'elasticnet', None],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear']},
             scoring='f1')

In [42]:
GCV_LRReg.best_params_

{'C': 10.0, 'class_weight': None, 'penalty': 'l2', 'solver': 'newton-cg'}

### Model score after Hyperparameter tuning

In [76]:
GCV_LRReg.score(scaled_test_set,y_test)

0.9264018691588786

## Decision Trees

In [38]:
DTClr = DecisionTreeClassifier()
DTClr.fit(scaled_train_set,y_train)

DecisionTreeClassifier()

In [115]:
DTClr.score(scaled_test_set,y_test)

0.8063139931740614

In [88]:
DTClr.get_params()

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': None,
 'max_leaf_nodes': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'random_state': None,
 'splitter': 'best'}

In [89]:
DTClrgrid = {"ccp_alpha":[0.0,1.0,3.5],
             "class_weight":[None,'balanced'],
             "criterion":['gini','entropy','log_loss'],
             "min_impurity_decrease":[0.0,2.1,1.1],
             "max_depth":[3,2,4,5,None],
             "max_features":[None,'auto','sqrt','log2'],
             "min_samples_split":[1,2,4,5],
             "min_samples_leaf":[1,2,4]
            }
RCV_DTClr = RandomizedSearchCV(estimator = DTClr,
                              param_distributions = DTClrgrid,
                              n_iter = 15,
                              scoring = 'f1',
                              n_jobs = -1,
                              cv = 5)
RCV_DTClr.fit(scaled_train_set,y_train)

RandomizedSearchCV(cv=5, estimator=DecisionTreeClassifier(), n_iter=15,
                   n_jobs=-1,
                   param_distributions={'ccp_alpha': [0.0, 1.0, 3.5],
                                        'class_weight': [None, 'balanced'],
                                        'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [3, 2, 4, 5, None],
                                        'max_features': [None, 'auto', 'sqrt',
                                                         'log2'],
                                        'min_impurity_decrease': [0.0, 2.1,
                                                                  1.1],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [1, 2, 4, 5]},
                   scoring='f1')

In [90]:
RCV_DTClr.best_params_

{'min_samples_split': 4,
 'min_samples_leaf': 1,
 'min_impurity_decrease': 1.1,
 'max_features': 'sqrt',
 'max_depth': 5,
 'criterion': 'gini',
 'class_weight': None,
 'ccp_alpha': 1.0}

In [92]:
RCV_DTClr.score(scaled_test_set,y_test)

0.8487229862475443

In [61]:
DTClrgrid_ = {"ccp_alpha":[0.0,1.0],
             "class_weight":['balanced',None],
             "criterion":['log_loss','entropy'],
             "min_impurity_decrease":[0.0],
             "max_depth":[2,4,None],
             "max_features":['sqrt','log2'],
             "min_samples_split":[2,4],
             "min_samples_leaf":[1,3]
            }
GCV_DTClr = GridSearchCV(estimator = DTClr,
                         param_grid = DTClrgrid_,
                         scoring = 'f1',
                         cv = 5,
                         )
GCV_DTClr.fit(scaled_train_set,y_train)

GridSearchCV(cv=5, estimator=DecisionTreeClassifier(),
             param_grid={'ccp_alpha': [0.0, 1.0],
                         'class_weight': ['balanced', None],
                         'criterion': ['log_loss', 'entropy'],
                         'max_depth': [2, 4, None],
                         'max_features': ['sqrt', 'log2'],
                         'min_impurity_decrease': [0.0],
                         'min_samples_leaf': [1, 3],
                         'min_samples_split': [2, 4]},
             scoring='f1')

In [62]:
GCV_DTClr.best_params_

{'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'entropy',
 'max_depth': 4,
 'max_features': 'sqrt',
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 3,
 'min_samples_split': 4}

In [71]:
GCV_DTClr.score(scaled_test_set,y_test)

0.9085475194261805

## Random Forest

In [7]:
RFClr = RandomForestClassifier()
RFClr.fit(scaled_train_set,y_train)

RandomForestClassifier()

In [8]:
RFClr.score(scaled_test_set,y_test)

0.8959044368600683

In [65]:
RFClr.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'sqrt',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 100,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [66]:
RFClrgrid = {"n_estimators": [10,100,150,200],
             "min_weight_fraction_leaf":[0.0,0.5,2.1],
             "max_leaf_nodes":[None,2,5,10],
             "criterion":['gini','entropy','log_loss'],
             "max_depth":[None,5,10],
             "max_features":['sqrt','log2',None],
             "min_samples_split":[1,2,4,7],
             "min_samples_leaf":[1,2,3]}
RCV_RFClr = RandomizedSearchCV(estimator = RFClr,
                              param_distributions = RFClrgrid,
                              n_iter = 20,
                              n_jobs = -1,
                              scoring = 'f1',
                              cv = 5)
RCV_RFClr.fit(scaled_train_set,y_train)

RandomizedSearchCV(cv=5, estimator=RandomForestClassifier(), n_iter=20,
                   n_jobs=-1,
                   param_distributions={'criterion': ['gini', 'entropy',
                                                      'log_loss'],
                                        'max_depth': [None, 5, 10],
                                        'max_features': ['sqrt', 'log2', None],
                                        'max_leaf_nodes': [None, 2, 5, 10],
                                        'min_samples_leaf': [1, 2, 3],
                                        'min_samples_split': [1, 2, 4, 7],
                                        'min_weight_fraction_leaf': [0.0, 0.5,
                                                                     2.1],
                                        'n_estimators': [10, 100, 150, 200]},
                   scoring='f1')

In [67]:
RCV_RFClr.best_params_

{'n_estimators': 200,
 'min_weight_fraction_leaf': 0.0,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_leaf_nodes': None,
 'max_features': 'sqrt',
 'max_depth': 10,
 'criterion': 'log_loss'}

In [68]:
RCV_RFClr.score(scaled_test_set,y_test)

0.9299363057324841

In [69]:
RFClrgrid_ = {"n_estimators": [10,200],
             "min_weight_fraction_leaf":[0.0],
             "max_leaf_nodes":[None],
             "criterion":['gini','entropy'],
             "max_depth":[None,10],
             "max_features":['log2',None],
             "min_samples_split":[1,4],
             "min_samples_leaf":[1,3]}

GCV_RFClr = GridSearchCV(estimator = RFClr,
                         param_grid = RFClrgrid_,
                         cv = 5,
                         n_jobs = -1,
                         scoring = 'f1')
GCV_RFClr.fit(scaled_train_set,y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': [None, 10],
                         'max_features': ['log2', None],
                         'max_leaf_nodes': [None], 'min_samples_leaf': [1, 3],
                         'min_samples_split': [1, 4],
                         'min_weight_fraction_leaf': [0.0],
                         'n_estimators': [10, 200]},
             scoring='f1')

In [76]:
GCV_RFClr.best_params_

{'criterion': 'entropy',
 'max_depth': None,
 'max_features': 'log2',
 'max_leaf_nodes': None,
 'min_samples_leaf': 1,
 'min_samples_split': 4,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200}

In [20]:
GCV_RFClr.score(scaled_test_set,y_test)

0.9279907084785134

## Support Vector Machines (SVM)

In [7]:
SVM = SVC().fit(scaled_train_set,y_train)

In [8]:
SVM.score(scaled_test_set,y_test)

0.8916382252559727

In [9]:
#get parameters
SVM.get_params()

{'C': 1.0,
 'break_ties': False,
 'cache_size': 200,
 'class_weight': None,
 'coef0': 0.0,
 'decision_function_shape': 'ovr',
 'degree': 3,
 'gamma': 'scale',
 'kernel': 'rbf',
 'max_iter': -1,
 'probability': False,
 'random_state': None,
 'shrinking': True,
 'tol': 0.001,
 'verbose': False}

In [10]:
#SVM
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'C': Real(0.01,50),
    "kernel":['linear','poly','rbf','sigmoid'],
}
clf = SVC(gamma='scale')
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=20, n_jobs=-1,scoring='accuracy')
Bayes.fit(scaled_train_set,y_train)

BayesSearchCV(cv=3, estimator=SVC(), n_iter=20, n_jobs=-1, scoring='accuracy',
              search_spaces={'C': Real(low=0.01, high=50, prior='uniform', transform='normalize'),
                             'kernel': ['linear', 'poly', 'rbf', 'sigmoid']})

In [11]:
Bayes.best_params_

OrderedDict([('C', 4.558419648506369), ('kernel', 'rbf')])

In [12]:
SVM_opt = SVC(kernel = 'rbf', C = 12.140873261631244)
SVM_opt.fit(scaled_train_set,y_train)

SVC(C=12.140873261631244)

In [13]:
SVM_opt.score(scaled_test_set,y_test)

0.8959044368600683

## Perceptron

In [106]:
PClr = Perceptron(tol=1e-3, random_state=0)
PClr.fit(scaled_test_set,y_test)

Perceptron()

In [120]:
PClr.score(scaled_test_set,y_test)

0.8626279863481229

In [108]:
PClr.get_params()

{'alpha': 0.0001,
 'class_weight': None,
 'early_stopping': False,
 'eta0': 1.0,
 'fit_intercept': True,
 'l1_ratio': 0.15,
 'max_iter': 1000,
 'n_iter_no_change': 5,
 'n_jobs': None,
 'penalty': None,
 'random_state': 0,
 'shuffle': True,
 'tol': 0.001,
 'validation_fraction': 0.1,
 'verbose': 0,
 'warm_start': False}

In [109]:
PClrgrid = {'max_iter':[1,10,100,1000,10000],
            'penalty':['l2','l1','elasticnet',None],
            'class_weight':['balanced',None],
            'alpha':[0.0001,0.01,0.1],
            'n_jobs':[-1]
           }
RCV_PClr = RandomizedSearchCV(estimator = PClr,
                              param_distributions = PClrgrid,
                              n_iter = 20,
                              scoring = 'f1',
                              n_jobs = -1,
                              cv = 5)
RCV_PClr.fit(scaled_train_set,y_train)

RandomizedSearchCV(cv=5, estimator=Perceptron(), n_iter=20, n_jobs=-1,
                   param_distributions={'alpha': [0.0001, 0.01, 0.1],
                                        'class_weight': ['balanced', None],
                                        'max_iter': [1, 10, 100, 1000, 10000],
                                        'n_jobs': [-1],
                                        'penalty': ['l2', 'l1', 'elasticnet',
                                                    None]},
                   scoring='f1')

In [110]:
RCV_PClr.best_params_

{'penalty': None,
 'n_jobs': -1,
 'max_iter': 1000,
 'class_weight': None,
 'alpha': 0.0001}

In [128]:
RCV_PClr.score(scaled_test_set,y_test)

0.8985163204747776

In [124]:
PClrgrid_ = {'max_iter':[10,100,1000],
            'penalty':['l2','l1',None],
            'class_weight':['balanced',None],
            'alpha':[0.0001,0.01],
            'n_jobs':[-1]
            }
GCV_PClr = GridSearchCV(estimator = PClr,
                         param_grid = PClrgrid_,
                         cv = 5,
                         n_jobs = -1,
                         scoring = 'f1')
GCV_PClr.fit(scaled_train_set,y_train)

GridSearchCV(cv=5, estimator=Perceptron(), n_jobs=-1,
             param_grid={'alpha': [0.0001, 0.01],
                         'class_weight': ['balanced', None],
                         'max_iter': [10, 100, 1000], 'n_jobs': [-1],
                         'penalty': ['l2', 'l1', None]},
             scoring='f1')

In [125]:
GCV_PClr.best_params_

{'alpha': 0.0001,
 'class_weight': None,
 'max_iter': 10,
 'n_jobs': -1,
 'penalty': 'l2'}

In [127]:
GCV_PClr.score(scaled_test_set,y_test)

0.8993448481238832

# Result 

In [ ]:
After considering all five conventional classification models, we come to conclusion that Random forest is the best model.

# Feature Selection

In [7]:
def RF(selected_features):
    RF = RandomForestClassifier(criterion = 'entropy',max_depth = None, max_features = 'log2', max_leaf_nodes = None,
                               min_samples_leaf = 1,min_samples_split = 4, min_weight_fraction_leaf = 0.0, n_estimators = 200)
    RF_result = cross_validation(RF, selected_features,y_test, 5)
    print(RF_result)

In [8]:
def cross_validation(model, _X, _y, _cv=5):
      _scoring = ['f1']
      results = cross_validate(estimator=model,
                               X=_X,
                               y=_y,
                               cv=_cv,
                               scoring=_scoring,
                               return_train_score=True)
      
      return {
              "Mean Validation F1 Score": results['test_f1'].mean()
              }

# Correlation-based feature selection (CFS)

In [9]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import mutual_info_classif
# X is your feature matrix, y is your target variable
def CFS(features):
    selector = SelectKBest(score_func=mutual_info_classif, k=features) # choose the number of features you want to keep
    CFS = selector.fit_transform(scaled_train_set, y_train)
    colu = list(X_test.columns[selector.get_support()])
    new_dataframe = X_test.filter(items=colu)
    print('The result for Random Forest')
    RF(new_dataframe)

In [19]:
for i in range(5,95,5):
    print("The number of features selected : ",i)
    CFS(i)

The number of features selected :  5
The result for Random Forest
{'Mean Validation F1 Score': 0.9039126941269409}
The number of features selected :  10
The result for Random Forest
{'Mean Validation F1 Score': 0.9094813444123309}
The number of features selected :  15
The result for Random Forest
{'Mean Validation F1 Score': 0.9108024001384303}
The number of features selected :  20
The result for Random Forest
{'Mean Validation F1 Score': 0.9202162630369564}
The number of features selected :  25
The result for Random Forest
{'Mean Validation F1 Score': 0.917534189668582}
The number of features selected :  30
The result for Random Forest
{'Mean Validation F1 Score': 0.917179367047105}
The number of features selected :  35
The result for Random Forest
{'Mean Validation F1 Score': 0.9195399994213134}
The number of features selected :  40
The result for Random Forest
{'Mean Validation F1 Score': 0.9198071678536894}
The number of features selected :  45
The result for Random Forest
{'Mean V

In [10]:
for i in range(90,94):
    print("The number of features selected : ",i)
    CFS(i)

The number of features selected :  90
The result for Random Forest
{'Mean Validation F1 Score': 0.9259616975502982}
The number of features selected :  91
The result for Random Forest
{'Mean Validation F1 Score': 0.9217534319812177}
The number of features selected :  92
The result for Random Forest
{'Mean Validation F1 Score': 0.9229386487163408}
The number of features selected :  93
The result for Random Forest
{'Mean Validation F1 Score': 0.9223045748646614}


In [143]:
#The max f1-score is for 90 features

# Principal component analysis (PCA)

In [39]:
from sklearn.decomposition import PCA

In [40]:
def PCA_sel(features):
    pca_sel = PCA(n_components = features)

    pct = pca_sel.fit_transform(X_test)
    RF(pct)
#     print(pct)

In [41]:
for i in range(5,95,5):
    print("The number of features selected : ",i)
    PCA_sel(i)
    

The number of features selected :  5
{'Mean Validation F1 Score': 0.8955510055281414}
The number of features selected :  10
{'Mean Validation F1 Score': 0.9105396409698894}
The number of features selected :  15
{'Mean Validation F1 Score': 0.9099626578535961}
The number of features selected :  20
{'Mean Validation F1 Score': 0.9112547442821001}
The number of features selected :  25
{'Mean Validation F1 Score': 0.9129278263755118}
The number of features selected :  30
{'Mean Validation F1 Score': 0.9090332511300627}
The number of features selected :  35
{'Mean Validation F1 Score': 0.9099051619807745}
The number of features selected :  40
{'Mean Validation F1 Score': 0.9157095498144342}
The number of features selected :  45
{'Mean Validation F1 Score': 0.9118611819686647}
The number of features selected :  50
{'Mean Validation F1 Score': 0.9120949508842486}
The number of features selected :  55
{'Mean Validation F1 Score': 0.907819471195746}
The number of features selected :  60
{'Mean 

In [ ]:
# The highest f-score is for 45 features

# L1 regularization

In [20]:
l1 = SelectFromModel(LogisticRegression(C=1, penalty='l1',solver='liblinear'))
l1.fit(scaled_train_set, y_train)

SelectFromModel(estimator=LogisticRegression(C=1, penalty='l1',
                                             solver='liblinear'))

In [21]:
l1.get_support()
colu = X_train.columns[l1.get_support()]
colu = list(colu)

In [22]:
new = X_test.filter(items=colu)
new.head()

,10Percentile,Energy,Entropy,InterquartileRange,Kurtosis,Maximum,MeanAbsoluteDeviation,Mean,Median,RobustMeanAbsoluteDeviation,...,LargeAreaHighGrayLevelEmphasis,LargeAreaLowGrayLevelEmphasis,SmallAreaEmphasis,SmallAreaHighGrayLevelEmphasis,SmallAreaLowGrayLevelEmphasis,Busyness,Coarseness,Complexity,Contrast.1,Strength
4160,-90.7,3198265,1.414487,18.00,2.567667,-40,11.043524,-72.710616,-73.0,7.756323,...,9.896353,0.251040,0.915347,6.965824,0.157760,0.803391,0.131118,0.254796,0.002927,0.565068
3417,-125.0,4583414,2.287683,234.00,1.095230,127,111.599841,9.464789,84.0,108.735571,...,89.650943,0.231531,0.920707,64.541317,0.158826,0.366062,0.039266,23.822087,0.347828,3.505375
1306,-114.0,4219550,2.439997,44.75,7.166517,126,37.534175,-64.281139,-77.0,18.772392,...,24.226843,0.140615,0.954369,20.935780,0.117925,0.399729,0.079289,6.602618,0.021005,5.511600
1792,-107.0,5211017,1.618324,23.00,34.251680,127,15.294723,-83.902549,-85.0,9.668807,...,14.299517,0.154904,0.947729,10.902802,0.124966,0.672917,0.052802,4.038046,0.005648,6.526806
2880,-112.0,4449091,2.368194,42.00,7.515471,127,36.058207,-63.371100,-76.0,17.808779,...,29.012635,0.154196,0.933426,18.989884,0.110968,0.698312,0.041129,15.558533,0.046723,2.419463


In [23]:
RF(new)

{'Mean Validation F1 Score': 0.920564397949059}


# Random forest feature importance

In [25]:
RF = RandomForestClassifier(criterion = 'entropy',max_depth = None, max_features = 'log2', max_leaf_nodes = None,
                               min_samples_leaf = 1,min_samples_split = 4, min_weight_fraction_leaf = 0.0, n_estimators = 200)
rfc_ = SelectFromModel(RF)
rfc_.fit(scaled_train_set, y_train)

SelectFromModel(estimator=RandomForestClassifier(criterion='entropy',
                                                 max_features='log2',
                                                 min_samples_split=4,
                                                 n_estimators=200))

In [32]:
colu =list(X_test.columns[rfc_.get_support()])
new_dataframe = X_test.filter(items = colu)
new_dataframe.head()

,Energy,Entropy,InterquartileRange,TotalEnergy,Uniformity,Information_Measure_Correlation2,InverseVariance,JointEnergy,JointEntropy,MaximumProbability,...,DependenceNonUniformity,GrayLevelNonUniformity,GrayLevelNonUniformity.1,GrayLevelNonUniformityNormalized,RunEntropy,RunLengthNonUniformity,GrayLevelNonUniformity.2,GrayLevelNonUniformityNormalized.1,SizeZoneNonUniformity,ZoneEntropy
4160,3198265,1.414487,18.00,3198265,0.415703,0.899130,0.160000,0.349867,1.907467,0.440000,...,390.154110,242.770548,213.299424,0.409404,1.963910,417.072937,213.299424,0.409404,417.072937,1.963910
3417,4583414,2.287683,234.00,4583414,0.256830,0.956150,0.152322,0.181203,3.245128,0.339623,...,241.118310,91.174648,79.345912,0.249515,2.835445,258.270440,79.345912,0.249515,258.270440,2.835445
1306,4219550,2.439997,44.75,4219550,0.217145,0.932916,0.394486,0.115771,3.670130,0.237288,...,446.807829,122.035587,113.196597,0.213982,2.779614,468.754253,113.196597,0.213982,468.754253,2.779614
1792,5211017,1.618324,23.00,5211017,0.377076,0.675749,0.276353,0.224660,2.594043,0.358209,...,511.977511,251.509745,232.909823,0.375056,1.994389,540.819646,232.909823,0.375056,540.819646,1.994389
2880,4449091,2.368194,42.00,4449091,0.224263,0.968533,0.248290,0.103253,3.798234,0.200000,...,435.633826,136.576355,124.833935,0.225332,2.828162,465.389892,124.833935,0.225332,465.389892,2.828162


In [33]:
RF(new_dataframe)

{'Mean Validation F1 Score': 0.9125115808198435}


# Mutual information

In [105]:
# https://github.com/anujdutt9/Feature-Selection-for-Machine-Learning/blob/master/Filter%20Methods/Mutual-Information.ipynb

In [95]:
mutual_info = mutual_info_classif(X_train, y_train)

In [34]:
for i in range(5,95,5):
    k_best_features = SelectKBest(mutual_info_classif, k=i).fit(X_train, y_train)
    colu =list(X_test.columns[k_best_features.get_support()])
    new_dataframe = X_test.filter(items = colu)
    print("The number of features selected : ", i)
    RF(new_dataframe)

The number of features selected :  5
{'Mean Validation F1 Score': 0.9017252334938597}
The number of features selected :  10
{'Mean Validation F1 Score': 0.9007222573891152}
The number of features selected :  15
{'Mean Validation F1 Score': 0.9106982456841484}
The number of features selected :  20
{'Mean Validation F1 Score': 0.9202802540731593}
The number of features selected :  25
{'Mean Validation F1 Score': 0.9175084080309045}
The number of features selected :  30
{'Mean Validation F1 Score': 0.9218996173236127}
The number of features selected :  35
{'Mean Validation F1 Score': 0.9199465499403479}
The number of features selected :  40
{'Mean Validation F1 Score': 0.9194502332343006}
The number of features selected :  45
{'Mean Validation F1 Score': 0.9211099391608141}
The number of features selected :  50
{'Mean Validation F1 Score': 0.9194683718955208}
The number of features selected :  55
{'Mean Validation F1 Score': 0.9211357174184075}
The number of features selected :  60
{'Mean

In [104]:
# The Max value is for 70 features 

# Recursive Feature Elimination (RFE)

In [16]:
from sklearn.feature_selection import RFE


In [17]:
def RFE_sel(features):
    sel = RFE(RandomForestClassifier(), n_features_to_select = features)
    sel.fit(X_train, y_train)
    colu = list(X_test.columns[sel.get_support()])
    new_dataframe = X_test.filter(items=colu)
    print('The result for Random Forest')
    RF(new_dataframe)

In [18]:
for i in range(5,95,5):
    print("The number of features selected : ",i)
    RFE_sel(i)
    

The number of features selected :  5
The result for Random Forest
{'Mean Validation F1 Score': 0.9051509213788457}
The number of features selected :  10
The result for Random Forest
{'Mean Validation F1 Score': 0.9166055165379522}
The number of features selected :  15
The result for Random Forest
{'Mean Validation F1 Score': 0.9202353962220734}
The number of features selected :  20
The result for Random Forest
{'Mean Validation F1 Score': 0.9194758658458072}
The number of features selected :  25
The result for Random Forest
{'Mean Validation F1 Score': 0.9166933717257081}
The number of features selected :  30
The result for Random Forest
{'Mean Validation F1 Score': 0.9183833894387071}
The number of features selected :  35
The result for Random Forest
{'Mean Validation F1 Score': 0.920599631294395}
The number of features selected :  40
The result for Random Forest
{'Mean Validation F1 Score': 0.9228942441788697}
The number of features selected :  45
The result for Random Forest
{'Mean 

In [36]:
colu =list(X_test.columns[sel.get_support()])
new_dataframe = X_test.filter(items = colu)
new_dataframe.head()

,Energy,Entropy,InterquartileRange,Mean,RootMeanSquared,Skewness,TotalEnergy,Uniformity,Variance,ClusterProminence,...,GrayLevelNonUniformityNormalized.1,GrayLevelVariance.2,HighGrayLevelZoneEmphasis,SizeZoneNonUniformity,SmallAreaEmphasis,ZoneEntropy,ZoneVariance,Busyness,Complexity,Strength
4160,3198265,1.414487,18.00,-72.710616,74.003251,0.181011,3198265,0.415703,189.647422,3.776562,...,0.409404,0.440501,7.497121,417.072937,0.915347,1.963910,0.129332,0.803391,0.254796,0.565068
3417,4583414,2.287683,234.00,9.464789,113.626693,-0.208926,4583414,0.256830,12821.443130,5365.604429,...,0.249515,20.141500,69.185535,258.270440,0.920707,2.835445,0.134261,0.366062,23.822087,3.505375
1306,4219550,2.439997,44.75,-64.281139,86.649271,2.132672,4219550,0.217145,3376.031281,1506.061458,...,0.213982,5.657670,21.551985,468.754253,0.954369,2.779614,0.062271,0.399729,6.602618,5.511600
1792,5211017,1.618324,23.00,-83.902549,88.389022,4.668298,5211017,0.377076,772.981508,48.598213,...,0.375056,1.435179,11.478261,540.819646,0.947729,1.994389,0.087911,0.672917,4.038046,6.526806
2880,4449091,2.368194,42.00,-63.371100,85.472616,2.215709,4449091,0.224263,3289.671808,3323.775231,...,0.225332,5.009527,20.722022,465.389892,0.933426,2.828162,0.118303,0.698312,15.558533,2.419463


In [37]:
RF(new_dataframe)

{'Mean Validation F1 Score': 0.9212763665837155}


In [ ]:
# https://github.com/krishnadulal/Feature-Selection-in-Machine-Learning-using-Python-All-Code